In [304]:
import os
os.environ["COHERE_API_KEY"] = 'U7e2tqNo5w6LcWgggi9RHyq26s7FPNnmy2OKiOED'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] ='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'

In [326]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#loading the documents from the folder
loader = PyPDFDirectoryLoader("data/")
docs = loader.load()
split_text=RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,
    chunk_overlap=0
)

split_doc=split_text.split_documents(docs)

In [327]:
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma

cembed=CohereEmbeddings(model="embed-english-v3.0")

vec_store=Chroma.from_documents(
    documents=split_doc,
    embedding=cembed
)


In [328]:
retriever = vec_store.as_retriever()

In [329]:
from pydantic import BaseModel, Field


#ROuter

In [330]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import BaseModel,Field

class search_web(BaseModel):
    """Use web_search for all general queries not covered in the documents."""
    query: str = Field(description="The query to search on the internet.")

class vector_store(BaseModel):
    """Use vectorstore for questions on Agentic AI."""
    query: str = Field(description="The query to retrieve from the RAG vectorstore.")



    

In [331]:
preamble = """You are a router. Direct the user query to the appropriate tool based on topic relevance.

Use the **vectorstore** if the user’s question relates to any of the following topics discussed in the documents:

- Agentic AI systems and architectures
- The role and future of Small Language Models (SLMs) in Agentic AI
- Economic, legal, and operational implications of deploying Agentic AI
- Prompt engineering, tool use, and modular AI agent design
- Adversarial attacks or robustness concerns in LLM-based agents

Use **web_search** if the question is unrelated to these domains or requires real-time, general, or external web information.

"""


In [332]:
llm = ChatCohere(model="command-r",temperature=0)

llm_router=llm.bind_tools(
    tools=[search_web,vector_store] , preamble=preamble
)

#This is testing purpose

In [333]:
prompt= ChatPromptTemplate.from_messages([("human", "{question}")])

In [334]:
ques_chain = prompt | llm_router

In [335]:
response = ques_chain.invoke(
    {"question": "Who owns the content made by agentic AI?"}
)
print(response.response_metadata["tool_calls"])



[{'id': 'vector_store_6gfw0t18qckh', 'type': 'function', 'function': {'name': 'vector_store', 'arguments': '{"query":"content made by agentic AI ownership"}'}}]


In [336]:
from typing import Literal


In [337]:
### Retrieval Grader


# Data model
class DocGrade(BaseModel):
    relevance_label: Literal["yes", "no"] = Field(
        description="Only respond with 'yes' or 'no' — is the document relevant to the question?"
    )



In [338]:
preamble = """
You are a grader assessing the relevance of a retrieved document to a user question.

Relevance means that the document either:
- Contains **keywords or phrases** from the question, OR
- Has **semantic similarity** in meaning or intent to the question.

You must assign a **binary relevance score**:
- Respond `"yes"` if the document directly addresses or answers the question.
- Respond `"no"` if it does not clearly relate to the question.

Respond with only `"yes"` or `"no"` — no explanations or other values are allowed.
"""


In [358]:

# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(DocGrade, preamble=preamble)

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "According to U.S. Copyright Office policy in 2023, can works generated solely by AI be copyrighted"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
response = retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)


relevance_label='yes'


In [359]:
print(doc_txt)

thereby creating significant ambiguity regarding the ownership of outputs from fully autonomous 
agentic AI systems.
A case in point is DABUS, an AI system that generated novel inventions (a food container 
and a flashing beacon). Patent applications naming DABUS as the inventor triggered legal battles 
around the world. Thus far, patent offices and courts in major jurisdictions (US, UK, EU) have 
rejected AI inventorship, insisting that inventors must be natural persons. For instance, the US 
Federal Circuit in Thaler v. Vidal (2022) affirmed that under current statutes, only humans can be 
inventors.2 The European Patent Office and UK Patent Office reached similar conclusions.3 
However, notable outliers exist: South Africa granted a patent with DABUS as inventor (albeit via a 
formality with no substantive examination).4 In Australia, the Federal Court (2021) initially ruled 
that AI could be an inventor under its law,5 but this was later overturned on appeal by the Full 
Federal Co

In [360]:
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

preamble = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."""


llm = ChatCohere(model="command-r",temperature=0).bind(preamble=preamble)

In [361]:
def prompt(x):
    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(
                f"Question: {x['question']} \nAnswer: ",
                additional_kwargs={"documents": x["documents"]},
            )
        ]
    )

In [362]:
gen_chain = prompt | llm | StrOutputParser()

generation = gen_chain.invoke({"documents": docs, "question": question})
print(generation)

According to current U.S. Copyright Office policy, works generated solely by AI cannot be copyrighted. The law insists that inventors must be natural persons. However, the ownership of AI-generated works is a subject of much debate, with several notable outliers in the patenting of such inventions.


In [ ]:
preamble = """
You are an assistant for question-answering tasks.

Answer the user's question based on your general knowledge only — do not assume access to any external documents or context.

Keep your answer concise and limited to a maximum of three sentences.
If you are unsure or lack enough information, say "I don't know."

"""

def prompt(x):
    return ChatPromptTemplate.from_messages(
        [HumanMessage(f"Question: {x['question']} \nAnswer: ")]
    )

llm_chain_fall_back = prompt | llm | StrOutputParser()

# Run
question_fall_back = "Hi how are you?"
generation_fall_back = llm_chain_fall_back.invoke({"question": question_fall_back})
print(generation_fall_back)


Hello! I'm doing well, thank you for asking. How can I assist you today?


In [370]:

class GradeHallucinations(BaseModel):
    """Binary score for hallucination presence in the generated answer."""
    binary_label: Literal["yes", "no"] = Field(
        description="Respond only with 'yes' if the answer is grounded in the facts, 'no' if it is not"
    )


preamble = """
You are a grader assessing whether a language model's answer is grounded in the given set of facts.

Respond:
- 'yes' → if the answer is fully supported by the facts.
- 'no' → if any part of the answer is not supported by the facts.

You MUST respond only with 'yes' or 'no'. Do not add any explanation or alternative labels.
"""


# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(
    GradeHallucinations, preamble=preamble
)

# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        # ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)
documents = "\n\n".join([doc.page_content for doc in docs])


hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": documents, "generation": generation})

GradeHallucinations(binary_label='yes')

In [371]:
print(documents)

thereby creating significant ambiguity regarding the ownership of outputs from fully autonomous 
agentic AI systems.
A case in point is DABUS, an AI system that generated novel inventions (a food container 
and a flashing beacon). Patent applications naming DABUS as the inventor triggered legal battles 
around the world. Thus far, patent offices and courts in major jurisdictions (US, UK, EU) have 
rejected AI inventorship, insisting that inventors must be natural persons. For instance, the US 
Federal Circuit in Thaler v. Vidal (2022) affirmed that under current statutes, only humans can be 
inventors.2 The European Patent Office and UK Patent Office reached similar conclusions.3 
However, notable outliers exist: South Africa granted a patent with DABUS as inventor (albeit via a 
formality with no substantive examination).4 In Australia, the Federal Court (2021) initially ruled 
that AI could be an inventor under its law,5 but this was later overturned on appeal by the Full 
Federal Co

In [375]:
print(question)

According to U.S. Copyright Office policy in 2023, can works generated solely by AI be copyrighted


In [376]:
print(generation)

According to current U.S. Copyright Office policy, works generated solely by AI cannot be copyrighted. The law insists that inventors must be natural persons. However, the ownership of AI-generated works is a subject of much debate, with several notable outliers in the patenting of such inventions.


In [379]:
class AnswerGrade(BaseModel):
    """
    Evaluates if the answer addresses the user’s question.
    """
    answer_relevant: Literal["yes", "no"] = Field(
        description="Respond with 'yes' if the answer addresses the question; 'no' if it does not."
    )


preamble = """
You are a grader checking if an answer responds to a user's question.

Respond:
- 'yes' if the answer clearly relates to or addresses the question, even if it's not perfect.
- 'no' if the answer is off-topic or unrelated.

Only reply with 'yes' or 'no'.
"""



llm = ChatCohere(model="command-r", temperature=0)
answer_llm = llm.with_structured_output(AnswerGrade, preamble=preamble)


answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_chain = answer_prompt | answer_llm
result = answer_chain.invoke({"question": question, "generation": generation})

In [380]:
print("=== QUESTION ===")
print(question)
print("=== GENERATION ===")
print(generation)
print("=== RESULT ===")
print(result.answer_relevant)


=== QUESTION ===
According to U.S. Copyright Office policy in 2023, can works generated solely by AI be copyrighted
=== GENERATION ===
According to current U.S. Copyright Office policy, works generated solely by AI cannot be copyrighted. The law insists that inventors must be natural persons. However, the ownership of AI-generated works is a subject of much debate, with several notable outliers in the patenting of such inventions.
=== RESULT ===
yes


In [381]:
os.environ['TAVILY_API_KEY'] ='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'


In [382]:

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

C:\Users\n07pa\AppData\Local\Temp\ipykernel_42000\224678106.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults()
